In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import os as os
from selenium import webdriver
import sys
import time
from webdriver_manager.chrome import ChromeDriverManager

# Scrape MLPS community links 

In [10]:
# URL of page to be scraped
url = 'http://www.mncompass.org/profiles/neighborhoods/minneapolis-saint-paul'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')

In [27]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('ul', class_="minneapolis-communities-listing")

#Declare lists to store community names and links
community=[]
community_link=[]

# Loop through returned results
for result in results:
    # Find all community data in a tags
    a_community = result.find_all('a')  
        
    # Error handling
    try:
        #Loop through the community a tags to extract community names and website links 
        for n in a_community:
        
        # Get name of community
            community_name=n.text
        
        # Identify and return link to community
            link = n['href']

        # Run only if community_name and link are available
            if (community_name and link):
                #Append to lists
                community.append(community_name)
                community_link.append(link)               

    except Exception as e:
        print(e)

In [28]:
#Create nighborhood household income dataframe
community_income_df=pd.DataFrame({"Community":community
                                    ,"Website_Link":community_link
                                    ,"IncomeLess35000_count":""
                                    ,"IncomeLess35000_percent":""
                                    ,"IncomeLess35to49_count":""
                                    ,"IncomeLess35to49_percent":""
                                    ,"IncomeLess50to74_count":""
                                    ,"IncomeLess50to74_percent":""
                                    ,"IncomeLess75to99_count":""
                                    ,"IncomeLess75to99_percent":""
                                    ,"Income100Plus_count":""
                                    ,"Income100Plus_percent":""
                                    ,"Median_Income_Total":""
                                  })

community_income_df.head()

,Community,Website_Link,IncomeLess35000_count,IncomeLess35000_percent,IncomeLess35to49_count,IncomeLess35to49_percent,IncomeLess50to74_count,IncomeLess50to74_percent,IncomeLess75to99_count,IncomeLess75to99_percent,Income100Plus_count,Income100Plus_percent,Median_Income_Total
0,Calhoun Isles,../communities/minneapolis/calhoun-isles,,,,,,,,,,,
1,Camden,../communities/minneapolis/camden,,,,,,,,,,,
2,Central,../communities/minneapolis/central,,,,,,,,,,,
3,Longfellow,../communities/minneapolis/longfellow,,,,,,,,,,,
4,Near North,../communities/minneapolis/near-north,,,,,,,,,,,


In [29]:
#Build the community website link
community_income_df['Website_Link'] = community_income_df['Website_Link'].str[3:]
community_income_df['Website_Link']= 'http://www.mncompass.org/profiles/'+community_income_df['Website_Link']
community_income_df.head()

,Community,Website_Link,IncomeLess35000_count,IncomeLess35000_percent,IncomeLess35to49_count,IncomeLess35to49_percent,IncomeLess50to74_count,IncomeLess50to74_percent,IncomeLess75to99_count,IncomeLess75to99_percent,Income100Plus_count,Income100Plus_percent,Median_Income_Total
0,Calhoun Isles,http://www.mncompass.org/profiles/communities/...,,,,,,,,,,,
1,Camden,http://www.mncompass.org/profiles/communities/...,,,,,,,,,,,
2,Central,http://www.mncompass.org/profiles/communities/...,,,,,,,,,,,
3,Longfellow,http://www.mncompass.org/profiles/communities/...,,,,,,,,,,,
4,Near North,http://www.mncompass.org/profiles/communities/...,,,,,,,,,,,


# Scrape Household Income for communities in MLPS

In [31]:
#Run loop to traverse through city_weather_df and populate required fields in df
print("Beginning Data Retrieval")    
print("-----------------------------")

for index, row in community_income_df.iterrows():
    
    #launch url
    url = row['Website_Link']
    
    try:
    
        # create a new Chrome session
        options = webdriver.ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument('--headless')
        driver = webdriver.Chrome('C:/path/to/chromedriver.exe',options=options)
        driver.implicitly_wait(30)
        driver.get(url)

        #After opening the url above, Selenium clicks the specific income link
        python_button = driver.find_element_by_id('householdIncomeLess35000') 
    
        #click above link
        python_button.click() 
    
        #Selenium hands the page source to Beautiful Soup
        soup_level1=bs(driver.page_source, 'lxml')
    
    
        print(f"Processing Record {row['Community']}")
        print(f"URL: {row['Website_Link']}")
        
        community_income_df.loc[index, 'IncomeLess35000_count'] = soup_level1.find('td', class_='count', id='householdIncomeLess35000').text
        community_income_df.loc[index, 'IncomeLess35000_percent'] = soup_level1.find('td', class_='percent', id='householdIncomeLess35000').text
        community_income_df.loc[index, 'IncomeLess35to49_count'] = soup_level1.find('td', class_='count', id='householdIncomeLess35to49').text
        community_income_df.loc[index, 'IncomeLess35to49_percent'] = soup_level1.find('td', class_='percent', id='householdIncomeLess35to49').text
        community_income_df.loc[index, 'IncomeLess50to74_count'] = soup_level1.find('td', class_='count', id='householdIncomeLess50to74').text
        community_income_df.loc[index, 'IncomeLess50to74_percent'] = soup_level1.find('td', class_='percent', id='householdIncomeLess50to74').text
        community_income_df.loc[index, 'IncomeLess75to99_count'] = soup_level1.find('td', class_='count', id='householdIncomeLess75to99').text
        community_income_df.loc[index, 'IncomeLess75to99_percent'] = soup_level1.find('td', class_='count', id='householdIncomeLess75to99').text
        community_income_df.loc[index, 'Income100Plus_count'] = soup_level1.find('td', class_='count', id='householdIncome100Plus').text
        community_income_df.loc[index, 'Income100Plus_percent'] = soup_level1.find('td', class_='percent', id='householdIncome100Plus').text        
        community_income_df.loc[index, 'Median_Income_Total'] = soup_level1.find('td', class_='count', id='medianHouseholdIncome').text        

        
    except Exception as e:
        print(e)

Beginning Data Retrieval
-----------------------------
Processing Record Calhoun Isles
URL: http://www.mncompass.org/profiles/communities/minneapolis/calhoun-isles
Processing Record Camden
URL: http://www.mncompass.org/profiles/communities/minneapolis/camden
Processing Record Central
URL: http://www.mncompass.org/profiles/communities/minneapolis/central
Processing Record Longfellow
URL: http://www.mncompass.org/profiles/communities/minneapolis/longfellow
Processing Record Near North
URL: http://www.mncompass.org/profiles/communities/minneapolis/near-north
Processing Record Nokomis
URL: http://www.mncompass.org/profiles/communities/minneapolis/nokomis
Processing Record Northeast
URL: http://www.mncompass.org/profiles/communities/minneapolis/northeast
Processing Record Phillips
URL: http://www.mncompass.org/profiles/communities/minneapolis/phillips
Processing Record Powderhorn
URL: http://www.mncompass.org/profiles/communities/minneapolis/powderhorn
Processing Record Southwest
URL: http:

In [32]:
#Display df
community_income_df.head()

,Community,Website_Link,IncomeLess35000_count,IncomeLess35000_percent,IncomeLess35to49_count,IncomeLess35to49_percent,IncomeLess50to74_count,IncomeLess50to74_percent,IncomeLess75to99_count,IncomeLess75to99_percent,Income100Plus_count,Income100Plus_percent,Median_Income_Total
0,Calhoun Isles,http://www.mncompass.org/profiles/communities/...,"\n 3,475\n ...",\n 19.8%\n ...,"\n 2,146\n ...",\n 12.2%\n ...,"\n 3,283\n ...",\n 18.7%\n ...,"\n 2,182\n ...","\n 2,182\n ...","\n 6,494\n ...",\n 36.9%\n ...,"\n $ 73,755\n ..."
1,Camden,http://www.mncompass.org/profiles/communities/...,"\n 3,916\n ...",\n 35.6%\n ...,"\n 1,889\n ...",\n 17.2%\n ...,"\n 2,103\n ...",\n 19.1%\n ...,"\n 1,271\n ...","\n 1,271\n ...","\n 1,817\n ...",\n 16.5%\n ...,"\n $ 51,761\n ..."
2,Central,http://www.mncompass.org/profiles/communities/...,"\n 8,478\n ...",\n 39.6%\n ...,"\n 1,718\n ...",\n 8.0%\n ...,"\n 3,066\n ...",\n 14.3%\n ...,"\n 2,112\n ...","\n 2,112\n ...","\n 6,021\n ...",\n 28.1%\n ...,"\n $ 46,190\n ..."
3,Longfellow,http://www.mncompass.org/profiles/communities/...,"\n 4,368\n ...",\n 32.6%\n ...,"\n 1,509\n ...",\n 11.3%\n ...,"\n 2,273\n ...",\n 17.0%\n ...,"\n 1,575\n ...","\n 1,575\n ...","\n 3,663\n ...",\n 27.4%\n ...,"\n $ 60,983\n ..."
4,Near North,http://www.mncompass.org/profiles/communities/...,"\n 5,002\n ...",\n 45.1%\n ...,"\n 1,540\n ...",\n 13.9%\n ...,"\n 1,802\n ...",\n 16.3%\n ...,"\n 1,166\n ...","\n 1,166\n ...","\n 1,574\n ...",\n 14.2%\n ...,"\n $ 57,634\n ..."


In [33]:
#Save data to a csv file
output_data_file = "resources/MLPS_Hsld_Income_by_Community.csv"
output_path = os.path.join(output_data_file)
community_income_df.to_csv(output_path, header=True, index=False)